In [97]:
import qtree
from qtensor.optimisation.TensorNet import QtreeTensorNet
from qtensor.optimisation.Optimizer import WithoutOptimizer, TamakiExactOptimizer, GreedyOptimizer, TamakiOptimizer
from qtensor import CircuitComposer, QtreeSimulator, TorchBuilder
from qtensor.OpFactory import TorchBuilder
from qtree.operators import Gate, ParametricGate
from qtensor.contraction_backends import TorchBackend

import torch
import torch.nn as nn
import torch.multiprocessing as mp

import numpy as np
import itertools
from functools import partial
import time

In [60]:
def bucket_elimination(buckets, process_bucket_fn,
                       n_var_nosum=0):

    n_var_contract = len(buckets) - n_var_nosum

    result = None
    for n, bucket in enumerate(buckets[:n_var_contract]):
        if len(bucket) > 0:
            tensor = process_bucket_fn(bucket)
            i = 0
            for used_tensor in bucket:
                used_tensor._data = None
                i+=1
            if len(tensor.indices) > 0:
                # tensor is not scalar.
                # Move it to appropriate bucket
                first_index = int(tensor.indices[0])
                buckets[first_index].append(tensor)
            else:   # tensor is scalar
                if result is not None:
                    result *= tensor
                else:
                    result = tensor
            del tensor
            torch.cuda.empty_cache()

    # form a single list of the rest if any
    rest = list(itertools.chain.from_iterable(buckets[n_var_contract:]))
    if len(rest) > 0:
        # only multiply tensors
        tensor = process_bucket_fn(rest, no_sum=True)
        if result is not None:
            result *= tensor
        else:
            result = tensor
    return result

qtree.optimizer.bucket_elimination = bucket_elimination


class M(Gate):
    name = 'M'
    _changes_qubits = (0, )
    """
    Measurement gate. This is essentially the identity operator, but
    it forces the introduction of a variable in the graphical model
    """
    @staticmethod
    def gen_tensor():
        return torch.tensor([[1, 0], [0, 1]])


class RandU(ParametricGate):
    name = 'Rand'
    _changes_qubits=(0, 1)

    @staticmethod
    def _gen_tensor(**parameters):
        return parameters['unitary']

    def __str__(self):
        return ("{}".format(self.name) +
                "({})".format(','.join(map(str, self._qubits)))
        )


class LocalRandomUnitaryComposer(CircuitComposer):

    def __init__(self, n_qubits, n_layers):
        self.n_qubits = n_qubits
        self.n_layers = n_layers
        super().__init__()

    def _get_builder(self):
        return self._get_builder_class()(self.n_qubits)

    def _get_builder_class(self):
        return TorchBuilder

    def random_circuit(self, unitaries):
        self.builder.reset()
        for qubit in range(self.n_qubits):
            self.apply_gate(M, qubit)
        for layer in range(self.n_layers):
            qubit = np.random.randint(self.n_qubits-1)
            self.apply_gate(RandU, qubit, qubit+1, unitary=unitaries[layer])
        return self.builder.circuit
    
    def name():
        return 'Local_Random'


'''Compose the circuit to evaluate the trace of the target circuit'''
class TraceEvaluationCircuitComposer(CircuitComposer):
    
    def __init__(self, n_qubits, target_name):
        self.n_target_qubits = n_qubits
        self.n_qubits = n_qubits*2
        self.target_name = target_name
        super().__init__(n_qubits*2)

    def _get_builder(self):
        return self._get_builder_class()(self.n_qubits)
    
    def _get_builder_class(self):
        return TorchBuilder

    def added_circuit(self):
        for target_qubit in range(self.n_target_qubits):
            control_qubit = target_qubit + self.n_target_qubits
            self.apply_gate(self.operators.H, control_qubit)
            self.apply_gate(self.operators.cX, control_qubit, target_qubit)

    '''Building circuit whose first amplitude is the expectation value of the measured circuit wrt to the cost_operator'''
    def expectation_circuit(self, circuit):
        self.builder.reset()
        self.added_circuit()
        first_part = self.builder.circuit
        self.builder.inverse()
        second_part = self.builder.circuit
        self.builder.reset()
        self.static_circuit = first_part + circuit + second_part
        self.expectation_circuit_initialized = True

    def name(self):
        return 'TraceEvaluation' + self.target_name

In [149]:
def random_unitary_generator(n_batch, n_layers, n_qubits):
    module = nn.Linear(1,1)
    module.unitary = nn.Parameter(torch.rand(n_batch, n_layers, 2**n_qubits, 2**n_qubits, dtype=torch.cfloat))
    orthmod = nn.utils.parametrizations.orthogonal(module, name='unitary')
    results = orthmod.unitary.reshape(n_batch, n_layers, 2,2,2,2).detach()
    return results

def get_val(com, trace, sim, unitary):
    peo = None
    circuit = com.random_circuit(unitary)
    trace.expectation_circuit(circuit)
    trace_circuit = trace.static_circuit
    result = sim.simulate_batch(trace_circuit, peo=peo)
    return result.detach()

def get_vals(n_qubits, n_layers, num):

    com = LocalRandomUnitaryComposer(n_qubits, 2*n_layers)
    trace = TraceEvaluationCircuitComposer(n_qubits, LocalRandomUnitaryComposer.name())
    sim = QtreeSimulator(backend=TorchBackend())
    get_val_map_fn = partial(get_val, com, trace, sim)
    pool = mp.Pool(num)
    #unitaries = [random_unitary_generator(1, 2*n_layers, 2)[0] for i in range(num)]
    unitaries = random_unitary_generator(num, 2*n_layers, 2)
    result = pool.map(get_val_map_fn, unitaries)
    return torch.tensor(result)



In [150]:
n_qubits = 5
n_layers = 5
num = 300

com = LocalRandomUnitaryComposer(n_qubits, 2*n_layers)
trace = TraceEvaluationCircuitComposer(n_qubits, LocalRandomUnitaryComposer.name())
sim = QtreeSimulator(backend=TorchBackend())

start = time.time()
print(get_val(com,trace,sim,random_unitary_generator(num, 2*n_layers, 2)[0]))
stop = time.time()
print('Time taken for a single evaluation: ', stop - start)
start = time.time()
print(get_vals(n_qubits, n_layers, num))
stop = time.time()
print('Time taken for multiprocess evaluation: ', stop - start)

tensor([0.0594+0.0131j])
Time taken for a single evaluation:  0.02789449691772461
tensor([-0.0164+4.7013e-02j,  0.0074-3.9642e-02j, -0.0155-1.3785e-02j,
         0.0114-1.8630e-02j,  0.0035-1.2216e-02j, -0.0274+5.0258e-02j,
        -0.0144-3.2542e-03j, -0.0533-1.3009e-02j,  0.0427+7.1787e-03j,
         0.0370+4.2212e-02j,  0.0348+1.0481e-02j, -0.0225-1.2134e-03j,
        -0.0287+2.2749e-02j,  0.0216-1.8072e-03j, -0.0176-2.8489e-02j,
         0.0124-2.7965e-03j,  0.0092+1.2419e-02j, -0.0018+2.2085e-02j,
        -0.0490+7.9088e-03j,  0.0079-1.8899e-02j, -0.0216-9.0330e-03j,
         0.0230-8.5591e-03j,  0.0498+2.1807e-02j, -0.0002-1.6271e-02j,
         0.0119+2.1262e-02j, -0.0023-1.3102e-02j,  0.0091-3.5412e-02j,
         0.0255+1.8234e-02j,  0.0058-1.3129e-02j,  0.0260+6.8780e-02j,
         0.0068-2.7503e-02j,  0.0213+1.6584e-02j,  0.0168-5.0449e-03j,
         0.0050-1.1509e-02j, -0.0358-4.0369e-02j, -0.0700+5.8693e-03j,
        -0.0493-1.4588e-02j,  0.0172+1.8158e-02j,  0.0086+4.7241e-

In [58]:
random_unitary()

tensor([[[[[ 4.6283e-01+0.2583j,  9.3755e-02-0.1243j],
           [-3.1231e-01+0.4509j, -4.8559e-01-0.3977j]],

          [[ 5.7190e-01+0.1681j, -1.7862e-01+0.6550j],
           [-6.1045e-02-0.3716j, -3.8188e-02+0.2011j]]],


         [[[ 1.6808e-02+0.2601j,  5.6429e-01+0.2415j],
           [ 7.0022e-01+0.0474j,  2.9101e-04-0.2504j]],

          [[ 3.4176e-01+0.4231j,  1.5193e-01-0.3391j],
           [-1.7713e-02+0.2538j,  5.2350e-01+0.4768j]]]],



        [[[[ 3.1406e-01+0.5493j, -5.7233e-01+0.1092j],
           [ 3.8516e-01-0.1358j,  6.0042e-02-0.2997j]],

          [[ 2.7468e-01+0.2142j,  6.3156e-01+0.0350j],
           [ 5.5483e-01-0.0292j,  3.0702e-01+0.2750j]]],


         [[[ 4.6447e-01+0.3430j,  3.0259e-01-0.1958j],
           [-4.6860e-01+0.4402j, -5.5917e-02-0.3467j]],

          [[ 5.8346e-02+0.3762j,  1.9117e-02-0.3608j],
           [-9.1831e-02-0.3205j, -6.1344e-01+0.4870j]]]],



        [[[[ 5.5658e-01+0.0866j, -4.0369e-01+0.3855j],
           [-6.8105e-02+0.1786j,  5.3